In [1]:
import numpy as np
import pandas as pd

In [2]:
import csv

In [3]:
import gurobipy as gp
from gurobipy import GRB

In [4]:
#makes a dictionary for parcels 
parcels = {}
with open ('Parcels.txt', mode = 'r') as csv_file:
    for line in csv_file:
        line.rstrip('\n')
        (key,val, val2) = line.split(',')
        parcels[int(key)] = [float(val), float(val2)]
parcels
#type(parcels)

{1: [37.519, 337671.0],
 2: [6.62, 19860.0],
 3: [24.126, 48252.0],
 4: [5.379, 80685.0],
 5: [12.252, 49008.0],
 6: [4.839, 14517.0],
 7: [30.92, 185520.0],
 8: [7.555, 30220.0],
 9: [2.043, 8172.0],
 10: [32.031, 0.0],
 11: [1.359, 5436.0],
 12: [12.181, 182715.0],
 13: [25.38, 152280.0],
 14: [37.921, 37921.0],
 15: [7.672, 107408.0],
 16: [34.06, 204360.0],
 17: [12.022, 180330.0],
 18: [10.932, 163980.0],
 19: [5.001, 25005.0],
 20: [30.481, 213367.0],
 21: [9.311, 46555.0],
 22: [3.005, 15025.0],
 23: [8.648, 43240.0],
 24: [3.298, 16490.0],
 25: [20.731, 849971.0],
 26: [2.527, 12635.0],
 27: [5.738, 40166.0],
 28: [3.198, 47970.0],
 29: [6.45, 19350.0],
 30: [7.208, 28832.0],
 31: [26.188, 104752.0],
 32: [14.007, 98049.0],
 33: [11.412, 22824.0],
 34: [18.264, 127848.0],
 35: [10.363, 155445.0],
 36: [9.684, 48420.0],
 37: [18.946, 113676.0],
 38: [13.08, 26160.0],
 39: [1.845, 11070.0],
 40: [1.74, 13920.0],
 41: [1.689, 25335.0],
 42: [6.633, 19899.0],
 43: [59.836, 837704.0

In [5]:
pars, areas, costs = gp.multidict(parcels) #turns dictionary into multidict, specifying what each column is

In [6]:
budg = 1000000 #sets the limit for the budget constraint

In [7]:
#makes list of tuples for adjacency--can't be dictionary because not unique keys
adjacents = []
with open ('Adjacency.txt', mode = 'r') as csv_file:
    adjacents = [tuple(line) for line in csv.reader(csv_file)]
for idx, (x,y) in enumerate(adjacents): #converting to integers
    adjacents[idx] = (int(x), int(y))
adjacents
#type(adjacents)

[(2, 51),
 (3, 50),
 (3, 55),
 (3, 60),
 (4, 5),
 (4, 6),
 (4, 48),
 (5, 4),
 (5, 54),
 (6, 4),
 (6, 47),
 (6, 56),
 (7, 57),
 (7, 58),
 (8, 60),
 (8, 66),
 (8, 68),
 (10, 13),
 (12, 63),
 (12, 73),
 (13, 10),
 (13, 16),
 (14, 17),
 (14, 19),
 (14, 21),
 (14, 22),
 (14, 24),
 (15, 17),
 (15, 18),
 (15, 23),
 (15, 75),
 (16, 13),
 (16, 20),
 (16, 76),
 (17, 14),
 (17, 15),
 (17, 19),
 (17, 23),
 (17, 64),
 (18, 15),
 (18, 73),
 (18, 75),
 (19, 14),
 (19, 17),
 (19, 22),
 (19, 23),
 (20, 16),
 (20, 76),
 (21, 14),
 (22, 14),
 (22, 19),
 (22, 23),
 (22, 24),
 (23, 15),
 (23, 17),
 (23, 19),
 (23, 22),
 (23, 26),
 (23, 27),
 (23, 75),
 (24, 14),
 (24, 22),
 (25, 77),
 (25, 78),
 (26, 23),
 (26, 27),
 (27, 23),
 (27, 26),
 (27, 28),
 (27, 32),
 (27, 75),
 (28, 27),
 (28, 31),
 (28, 32),
 (28, 33),
 (29, 31),
 (29, 76),
 (31, 28),
 (31, 29),
 (31, 33),
 (31, 35),
 (31, 37),
 (31, 76),
 (32, 27),
 (32, 28),
 (32, 33),
 (32, 38),
 (33, 28),
 (33, 31),
 (33, 32),
 (33, 35),
 (33, 38),
 (33, 82)

In [8]:
m = gp.Model('ParcelModel') #starts a model "m"

Academic license - for non-commercial use only - expires 2021-06-21
Using license file /Users/zrand/gurobi.lic


In [9]:
x = m.addVars(pars, obj = areas, vtype = GRB.BINARY, name = "X") 
#sets decision variable X, binary, indexed by parcel ID,with area as the coefficient in the objective function 

In [10]:
y = m.addVars(adjacents, vtype = GRB.BINARY, name = "Y") #sets variable Y, binary, indexed by [i,j] in adjacents file

In [11]:
B = m.addConstr(sum(costs[i]*x[i]for i in pars) <= budg, name = "budget") #budget constraint

In [12]:
core = m.addConstr(x[23] == 1, name = "core") #parcel 23 must be included in the reserve

In [13]:
for j in parcels:
    A = [item[0] for item in adjacents  #finds all parcels adjacent to parcel j
            if item[1]==j]
    if not A:  #skips this constraint if there are no parcels adjacent to j
        pass
    else:
        m.addConstr(sum(y[i,j] for i in A) <= len(A)*x[j], name = "no_flow" + str(j)) #no flow for parcel J constraint

In [14]:
for i in parcels:
    A = [item[1] for item in adjacents  #finds all parcels adjacent to parcel i
            if item[0]==i]
    if not A:
        pass  #skips this constraint if there are no parcels adjacent to i
    else:
        m.addConstr(sum(y[i,j] for j in A) <= x[i], name = "one_flow" + str(i)) #one flow constraint for each parcel i 

In [15]:
 #sum of all flow variables is 1 less than the sum of all parcels selected
connected = m.addConstr(y.sum()== x.sum() - 1, name = "connected")

In [16]:
z = m.addVars(adjacents, vtype = GRB.INTEGER, name = "Z") #tail length contribution variable
w = m.addVars(parcels, vtype = GRB.INTEGER, name = "W") #tail length variable

In [17]:
M = len(parcels) #set the value of M in tail lenght constraint (not sure about this)

In [18]:
#tail length functions
for i in parcels:
    A = [item[1] for item in adjacents #finds all parcels adjacent to i
            if item[0]==i]
    if not A: #if there are no parcels adjacent to i, skips this constraint
        pass
    else:
         for j in A:
            if j != i: #for each j where j does not equal i, calculates tail contribution
                m.addConstr(z[i,j] >= w[i] + 1 - M*(1 - y[i,j]), name = "tail" + str(i)+str(j))

In [19]:
#tail length W
for j in parcels:
    A = [item[0] for item in adjacents #finds all parcels adjacent to j
            if item[1] == j]
    if not A:
        pass #skips this constraint if there are no parcels adjacent to j
    else:
        m.addConstr(w[j] == z.sum('*',j), name = "wtail" + str(j)) #calculates w[j]

In [20]:
m.setObjective(x.prod(areas), GRB.MAXIMIZE) #set objective function to maximize

In [21]:
m.write('ParcelModel.lp') #write lp file